## 大模型答案生成（RAG-Generation）

大模型根据检索内容生成的原理：把检索到的【知识库内容】、【用户问题】、【回答要求】一起给大模型，让大模型按照你的要求来回答

<img src="/data/xieyu/Teaching/RAG/imgs/rag-in-action.jpeg" alt="向量检索流程图" width="60%" style="display: block; margin: auto;">

## 基础实现

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 初始化RAG工作流
template_rag = """
你是一个知识库回答专家，请你根据用户的问题，从知识库中找到最相关的信息，整理抽取出对应的详细信息，要求回复简洁，每次回答后提醒我要检查我的回答是否正确，如果回答错误，请重新回答

检索到的内容：{context}
用户问题：{query}
    """
## 这里使用了langchain的lcel语法，如果没有学习过的可以取查看langchain的官方文档
gpt = ChatOpenAI(model="gpt-4o-eus2", api_key="your_key", base_url="your_base_url")
rag_template= PromptTemplate.from_template(template_rag)
rag = {"context": RunnablePassthrough(), "query": RunnablePassthrough()} | rag_template | gpt

# 配置embeding模型
embeding_model_path = "/data/lilk/RAG/models/embeding/bge-m3"
# 配置运行的参数
embeding_model_kwargs = {"device": "cuda"}
embeding_encode_kwargs = {"normalize_embeddings": True}
embeding_model = HuggingFaceEmbeddings(model_name = embeding_model_path,
                                       model_kwargs=embeding_model_kwargs,
                                       encode_kwargs=embeding_encode_kwargs)
vector_store = FAISS.load_local(index_name="rag_teach",folder_path="/data/xieyu/Teaching/RAG/data_retriever/rag_teach",embeddings=embeding_model,allow_dangerous_deserialization=True )

# 执行检索
rag_content = vector_store.similarity_search("RAG是什么？")
result = rag.invoke({"context": rag_content, "query": "RAG是什么？"}).content
print(result)

RAG（Retrieval-Augmented Generation）是一种能让大量知识“活”起来，人与知识无障碍对话的技术。它的背景是，虽然AI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练它们的知识主要是互联网过往的公开内容，因此当我们有非公开的知识需要AI帮忙解读、总结时，这些大模型会遇到知识盲区。RAG技术的作用是让这些大模型能够了解到我们的非公开知识，从而更加精准地回答我们的问题。

请检查您的回答是否正确，如果回答错误，请重新回答。


## 代码优化

In [10]:
# 使用RunnableParallel来优化检索和回答的流程
# 将检索和回答合并为一个完整的RAG工作流

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
# 设置并行运行的检索和问题传递
setup_and_retrieval = RunnableParallel(
    {"context": vector_store.as_retriever(), 
     "query": RunnablePassthrough()}
)

# 构建完整的RAG链
rag_chain = setup_and_retrieval | rag_template | gpt 

# 执行RAG查询
result = rag_chain.invoke("RAG是什么？")
print(result)

content='RAG是一种能让大量知识“活”起来，人与知识无障碍对话的技术。随着AI大模型的发展，它们已经能够与用户流畅交流，解读知识并总结文章。但是，由于这些模型的训练数据主要来源于互联网的公开内容，当涉及到非公开知识时，AI模型会遇到知识盲区。因此，RAG技术的目标是让AI模型能够获取并解读非公开知识，从而更加精准地回答用户的问题。\n\n请检查我的回答是否正确，如果回答错误，请重新回答。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 742, 'total_tokens': 859}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None} id='run-d745f390-41a0-40cb-994d-24e2dad61d83-0' usage_metadata={'input_tokens': 742, 'output_tokens': 117, 'total_tokens': 859}


## 知识库记忆功能

有时候我们提问知识库，但是希望多轮回答，则需要让这个知识库带有记忆功能

In [24]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory

def get_rephrase_prompt():
    system_prompt = """\
    鉴于以下对话和后续问题，将后续问题重新表述为独立问题。
    """
    rephrase_prompt = ChatPromptTemplate([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    return rephrase_prompt

def get_rag_prompt():
    system_prompt = """\
    你是一个知识库回答专家，请你根据用户的问题，从知识库中找到最相关的信息，整理抽取出对应的详细信息，要求回复简洁，每次回答后提醒我要检查我的回答是否正确，如果回答错误，请重新回答
    知识库内容如下：
    {context}
    """
    rag_template = ChatPromptTemplate([
        MessagesPlaceholder("chat_history"),
    ("system", system_prompt),
        ("human", "{input}")
    ])
    return rag_template

rephrase_template = get_rephrase_prompt()
rag_template = get_rag_prompt()

# 定义带历史记录的检索器
history_aware_retriever = create_history_aware_retriever(gpt, vector_store.as_retriever(), rephrase_template)
rag_chain = create_stuff_documents_chain(gpt, rag_template)
# 创建检索链
rag_history_chain = create_retrieval_chain(history_aware_retriever, rag_chain)
# 添加消息历史记录功能 
store = {}
def get_session_history(session_id:str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 创建带历史记录的RAG链
conversational_rag_chain = RunnableWithMessageHistory(
    rag_history_chain,
    get_session_history, 
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)
# 测试带记忆的RAG
result = conversational_rag_chain.invoke(
    {"input": "RAG是什么?"}, 
    config={"configurable": {"session_id": "test123"}}
)
print(result["answer"])
print("-"*100)
# 测试第二个问题,会带上上下文
result = conversational_rag_chain.invoke(
    {"input": "它有什么优点?"}, 
    config={"configurable": {"session_id": "test123"}}
)
print(result["answer"])

RAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术。AI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题。

请检查我的回答是否正确，如果回答错误，请重新回答。
----------------------------------------------------------------------------------------------------
RAG技术有以下优点：

1. **知识活化**：RAG可以让大量知识“活”起来，使人与知识无障碍对话，提供更加流畅的交流体验。
2. **精准回答**：通过结合非公开知识，RAG技术能够更精准地回答用户的问题，解决AI大模型在非公开知识上的盲区。
3. **多场景应用**：RAG技术可以应用在企业知识库、客服机器人、产品自动推荐等多个场景中。
4. **高效检索**：RAG可以通过多种方式（如大模型改写、向量检索等）高效地检索所需信息，提升查询效率。
5. **灵活集成**：RAG技术的多种应用方案（如langchain、dbgpt、ragflow等）使其能够灵活集成到不同的系统中，满足多样化的需求。

请检查我的回答是否正确，如果回答错误，请重新回答。


## 总结


1. 大模型答案生成，需要把【知识库内容】、【用户问题】、【回答要求】一起给大模型，让大模型按照你的要求来回答
2. 大模型答案生成，可以使用langchain的lcel语法，也可以使用langchain的RunnablePassthrough来优化检索和回答的流程
3. 大模型答案生成，可以使用RunnableParallel来优化检索和回答的流程
4. 大模型答案生成，可以使用RunnableWithMessageHistory来实现知识库记忆功能

很多小伙伴看到这里会开始疑惑说那如果知识库需要联网呢？
不知道大家有没有思考过一个问题？如果网络上也有你想要提问的问题的答案呢？但是跟你的知识库存在多个版本？这种情况应该以哪个为主呢？这就会产生很多回答准确度的问题，所以你如果你对这个感兴趣就可以再看看下一张，rag_agent，会有更多方案进行介绍